In [1]:
%run ./ProlificAcademic_CrisisLogger_data_processing.ipynb #processed_transcriptions and specify_positive_data_by_survey variables
#hold the crisislogger and prolific academic data, respectively.
%run ./FocusGroup_data_processing.ipynb #all_focusgroup_processed_text holds the Focus group data.

c:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3209: DtypeWarning: Columns (93) have mixed types.Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
c:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3209: DtypeWarning: Columns (5,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [3]:
import tomotopy as tp

In [4]:
def collapse_list_of_strings(list_of_strings):
    return ' '.join(list_of_strings)

processed_transcriptions_collapsed=[collapse_list_of_strings(answer) for answer in processed_transcriptions[0]]

In [31]:
mdl = tp.LDAModel(k=5, min_df=5, rm_top=10)
for line in range(len(processed_transcriptions[0])):
    mdl.add_doc(processed_transcriptions[0][line])
    
mdl.train()

for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))
    
print(len(mdl.docs))
print(np.argmax(mdl.infer(mdl.docs[0])[0]))
mdl.infer(mdl.docs[0])[0]

Top 10 words of topic #0
[('never', 0.031088031828403473), ('student', 0.02808048203587532), ('miss', 0.027077965438365936), ('bad', 0.0260754507035017), ('fear', 0.0260754507035017), ('virus', 0.025072934105992317), ('teacher', 0.024070417508482933), ('struggle', 0.024070417508482933), ('normal', 0.024070417508482933), ('safe', 0.018055319786071777)]
Top 10 words of topic #1
[('look', 0.026512883603572845), ('together', 0.02196911722421646), ('keep', 0.02196911722421646), ('every', 0.02045452781021595), ('hope', 0.01969723403453827), ('thank', 0.018182644620537758), ('mental', 0.017425350844860077), ('activity', 0.016668055206537247), ('health', 0.016668055206537247), ('husband', 0.016668055206537247)]
Top 10 words of topic #2
[('big', 0.02600891701877117), ('put', 0.02408304437994957), ('online', 0.023120107129216194), ('difficult', 0.023120107129216194), ('okay', 0.022157171741127968), ('use', 0.021194234490394592), ('lose', 0.020231297239661217), ('challenge', 0.01926836185157299),

array([0.21379074, 0.03856613, 0.19451872, 0.39665538, 0.15646905],
      dtype=float32)

In [4]:
def identify_additional_stopwords_with_LDA(tokenized_text_list, number_of_topics, stopword_threshold):
    common_dictionary = Dictionary(tokenized_text_list)
    common_corpus = [common_dictionary.doc2bow(text) for text in tokenized_text_list]
    lda = LdaModel(common_corpus, num_topics=number_of_topics, id2word=common_dictionary, alpha='auto', eta='auto')
    most_common_words=Counter([term[0] for topic_num in range(number_of_topics) 
                                       for term in lda.show_topic(topicid=topic_num)]).most_common()
    additional_stopwords=[term[0] for term in most_common_words if term[1]>=stopword_threshold]
    return additional_stopwords

In [5]:
identify_additional_stopwords_with_LDA(processed_transcriptions[0], 5, 3)

['life', 'social', 'right']

In [6]:
common_dictionary = Dictionary(processed_transcriptions[0])
common_corpus = [common_dictionary.doc2bow(text) for text in processed_transcriptions[0]]
lda = LdaModel(common_corpus, num_topics=5, id2word=common_dictionary, alpha='auto', eta='auto')

In [7]:
lda.show_topics()

[(0,
  '0.007*"happen" + 0.007*"right" + 0.005*"look" + 0.005*"good" + 0.005*"come" + 0.005*"never" + 0.004*"life" + 0.004*"big" + 0.004*"month" + 0.004*"many"'),
 (1,
  '0.007*"good" + 0.005*"stay" + 0.005*"crisis" + 0.005*"hope" + 0.005*"learn" + 0.004*"life" + 0.004*"son" + 0.004*"social" + 0.004*"share" + 0.004*"around"'),
 (2,
  '0.006*"learn" + 0.005*"hard" + 0.005*"little" + 0.005*"come" + 0.004*"new" + 0.004*"difficult" + 0.004*"social" + 0.004*"crisis" + 0.004*"together" + 0.004*"life"'),
 (3,
  '0.007*"something" + 0.007*"son" + 0.005*"still" + 0.005*"hard" + 0.004*"together" + 0.004*"miss" + 0.004*"learn" + 0.004*"fear" + 0.004*"social" + 0.004*"end"'),
 (4,
  '0.005*"hope" + 0.005*"health" + 0.005*"parent" + 0.005*"big" + 0.004*"never" + 0.004*"life" + 0.003*"last" + 0.003*"good" + 0.003*"live" + 0.003*"social"')]

In [8]:
topic_distribution=[lda.get_document_topics(bow=bow_text)[0][0] for bow_text in common_corpus]
print(Counter(topic_distribution))

Counter({0: 51, 1: 32, 2: 30, 3: 15, 4: 12})


In [5]:
Gaming_group_data=[text for gaming_group_text in all_focusgroup_processed_text[11:15] for text in gaming_group_text[0]]
#Gaming_group_data=[text for text in all_focusgroup_processed_text[0][0]]
common_dictionary = Dictionary(Gaming_group_data)
common_corpus = [common_dictionary.doc2bow(text) for text in Gaming_group_data]
lda = LdaModel(common_corpus, num_topics=5, id2word=common_dictionary, alpha='auto', eta='auto')

In [6]:
identify_additional_stopwords_with_LDA(Gaming_group_data, 5, 3)

['kid', 'one', 'lot', 'even']

In [7]:
topic_distribution=[lda.get_document_topics(bow=bow_text)[0][0] for bow_text in common_corpus]
print(Counter(topic_distribution))
lda.show_topics()

Counter({2: 85, 0: 54, 4: 46, 3: 32, 1: 11})


[(0,
  '0.012*"use" + 0.012*"friend" + 0.011*"phone" + 0.010*"pandemic" + 0.009*"guy" + 0.008*"even" + 0.008*"year" + 0.008*"00" + 0.008*"one" + 0.007*"kid"'),
 (1,
  '0.008*"kid" + 0.008*"1" + 0.007*"lot" + 0.006*"im" + 0.005*"two" + 0.005*"still" + 0.005*"open" + 0.005*"social" + 0.005*"three" + 0.005*"one"'),
 (2,
  '0.013*"back" + 0.011*"work" + 0.011*"kid" + 0.009*"lot" + 0.009*"need" + 0.008*"one" + 0.008*"social" + 0.008*"something" + 0.008*"feel" + 0.008*"pandemic"'),
 (3,
  '0.013*"take" + 0.011*"kid" + 0.011*"back" + 0.009*"one" + 0.009*"room" + 0.008*"even" + 0.008*"lot" + 0.008*"need" + 0.007*"people" + 0.007*"use"'),
 (4,
  '0.020*"phone" + 0.012*"00" + 0.012*"use" + 0.011*"lot" + 0.009*"one" + 0.009*"pandemic" + 0.009*"social" + 0.008*"medium" + 0.008*"even" + 0.008*"good"')]

In [13]:
additional_stopwords_list

[['like',
  'get',
  'school',
  'hour',
  'day',
  'even',
  'think',
  'thing',
  'way',
  'know',
  'year',
  'week',
  'really',
  'one'],
 ['like',
  'get',
  'school',
  'hour',
  'day',
  'even',
  'think',
  'thing',
  'way',
  'know',
  'year',
  'week',
  'really',
  'one'],
 ['like',
  'get',
  'school',
  'hour',
  'day',
  'even',
  'think',
  'thing',
  'way',
  'know',
  'year',
  'week',
  'really',
  'one'],
 ['like',
  'get',
  'school',
  'hour',
  'day',
  'even',
  'think',
  'thing',
  'way',
  'know',
  'year',
  'week',
  'really',
  'one'],
 ['use', 'two'],
 ['use', 'two'],
 ['use', 'two'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [219]:
topic_distribution=[lda.get_document_topics(bow=bow_text)[0][0] for bow_text in common_corpus if lda.get_document_topics(bow=bow_text)[0][1]>0.9]
Counter(topic_distribution)
topic_to_print=0
Gaming_group_original_text=[text for gaming_group_text in all_focusgroup_processed_text[0:4] for text in gaming_group_text[1]]
text_to_print=[original_text for bow_text, original_text, topic in zip(common_corpus, Gaming_group_original_text, topic_distribution) if lda.get_document_topics(bow=bow_text)[0][1]>0.9 and topic==topic_to_print]

In [220]:
[print(text, '\n') for text in text_to_print]

Hi everybody. My name is Parent 3 and I have an 11-year-old daughter and an 8-year-old son. We don\t watch television together at all, it\s just for me. They Roblox with their friends and that\s all they do. I\ve watched a couple of foreign things my friends have recommended, but I\d love to recommend a series, it\s long but it\s amazing, it\s called Orphan Black. It\s been around for a little while, it\s produced by the BBC. It\s sci-if and it\s about cloning. It has five seasons. It\s a real deep dive, but it\s incredible and now it just won an Emmy. So I\ll say Orphan Black, it\s really great. 

Yeah, so I have a similar experience with my eight-year-old. Pre-pandemic he was still in second grade so the use of computers was very limited. Maybe once a week he would type, words of the week was one of the options I picked to do. That was his only interaction with computer. Of course he had devices to play on, and he would go to his father\s house every other weekend and he would be mor

[None, None, None, None]

In [189]:
lda.get_document_topics(common_corpus[0])

[(2, 0.9812123)]

In [221]:
Gaming_group_data=[collapse_list_of_strings(text) for gaming_group_text in all_focusgroup_processed_text[0:4]
                                                  for text in gaming_group_text[0]]

In [222]:
number_of_tokens_per_text=[len(gaming_group_text[0]) for gaming_group_text in all_focusgroup_processed_text[0:4]]
date_list=['4/1/2020', '5/1/2020', '11/1/2020', '4/1/2021']
date_list_long=[list(repeat(date_list[i], number_of_tokens_per_text[i])) for i in range(4)]
date_list_long=[date for date_list in date_list_long for date in date_list]

In [223]:
topic_model=BERTopic(language="english", calculate_probabilities=True, verbose=True, n_gram_range=(1,1))
topics, probs = topic_model.fit_transform(Gaming_group_data)

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

2021-11-28 16:39:53,786 - BERTopic - Transformed documents to Embeddings
2021-11-28 16:39:57,956 - BERTopic - Reduced dimensionality with UMAP
2021-11-28 16:39:58,025 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [224]:
topics_over_time = topic_model.topics_over_time(Gaming_group_data, topics, date_list_long)

4it [00:01,  2.72it/s]


In [225]:
topic_model.visualize_topics_over_time(topics_over_time)

In [226]:
freq = topic_model.get_topic_info(); print(freq.head(10))
topic_model.visualize_barchart(topics=list(range(0,3)), n_words=10)

   Topic  Count                                  Name
0      0    204                0_like_school_kid_game
1     -1     21  -1_technology_pandemic_positive_game
2      1     17                   1_watch_name_kid_tv
3      2     15  2_experience_group_email_information


In [89]:
len(Gaming_group_data)

257